In [4]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import LSTM

inputs=Input(shape=(10,))

In [2]:
hid1=Dense(32, activation='relu')(inputs)
hid2=Dense(32, activation='relu')(hid1)
output=Dense(1, activation='sigmoid')(hid2)

model=Model(inputs=inputs, outputs=output)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', matrics=['accuracy'])

W0823 09:22:04.022691 13060 deprecation_wrapper.py:119] From C:\Users\Kyujin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0823 09:22:04.182915 13060 deprecation_wrapper.py:119] From C:\Users\Kyujin\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0823 09:22:04.245092 13060 deprecation_wrapper.py:119] From C:\Users\Kyujin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [5]:

inputs=Input(shape=(30,1))
lstm_layer=LSTM(10)(inputs)
x=Dense(10, activation='relu')(lstm_layer)
output=Dense(1, activation='sigmoid')(x)
Model(inputs=inputs, outputs=output)

<tf.Tensor 'lstm_1/TensorArrayReadV3:0' shape=(?, 10) dtype=float32>

In [6]:
from keras import Sequential


In [8]:
from keras.layers import SimpleRNN
model=Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [18]:
import numpy as np
timesteps=10 # 시점, 문장을 구성하는 단어의 길이
input_dim=4 # 단어가 4개인 벡터
hidden_size=8 #은닉상태의 크키
inputs=np.random.random((timesteps, input_dim))
inputs

hidden_state_t=np.zeros((hidden_size, ))

wx=np.random.random((hidden_size, input_dim))
print(wx.shape)
wh=np.random.random((hidden_size, hidden_size))
#(8,8)
#wh: 은닉상태의 크기* 은닉상태의 크기
b=np.random.random((hidden_size))#8

(8, 4)


In [20]:
total_hidden_states=[]
for input_t in inputs:
    output_t=np.tanh(np.dot(wx,input_t)+np.dot(wh,hidden_state_t)+b)
    #wx*wt+wh*ht-1+b
    total_hidden_states.append(list(output_t))

In [21]:
text="""멀티캠퍼스에 있는 교육생들이 공부하고 있다\n
교육생들은 딥러닝을 하고있다\n
결석한 교육생도 있고 지각하는 교육생도 있다"""

In [22]:
from keras_preprocessing.text import Tokenizer
t=Tokenizer()
t.fit_on_texts([text])
print(t.texts_to_sequences([text]))
print(t.word_index)

[[3, 4, 5, 6, 1, 7, 8, 9, 10, 2, 11, 12, 2, 1]]
{'있다': 1, '교육생도': 2, '멀티캠퍼스에': 3, '있는': 4, '교육생들이': 5, '공부하고': 6, '교육생들은': 7, '딥러닝을': 8, '하고있다': 9, '결석한': 10, '있고': 11, '지각하는': 12}


In [23]:
vocab_size=len(t.word_index)+1

In [43]:
vocab_size

13

In [44]:
sequences=list()
for line in text.split('\n'):
    encoded=t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence=encoded[:i+1]
        sequences.append(sequence)
#         print(sequences)
max_len=max(len(s) for s in sequences)# 최대 길이 문장

In [45]:
from keras_preprocessing.sequence import pad_sequences
sequences=pad_sequences(sequences, maxlen=max_len, padding='pre')#default

In [46]:
x=sequences[:,:-1]
y=sequences[:,-1]

In [47]:
from keras.utils import to_categorical
from keras.layers import *
y=to_categorical(y,num_classes=vocab_size)

In [48]:
y

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [49]:
model=Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x,y, epochs=300, verbose=2)

W0823 13:45:05.866078 13060 deprecation.py:323] From C:\Users\Kyujin\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0823 13:45:06.484088 13060 deprecation_wrapper.py:119] From C:\Users\Kyujin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/300
 - 1s - loss: 2.5762 - acc: 0.0909
Epoch 2/300
 - 0s - loss: 2.5655 - acc: 0.1818
Epoch 3/300
 - 0s - loss: 2.5564 - acc: 0.1818
Epoch 4/300
 - 0s - loss: 2.5469 - acc: 0.3636
Epoch 5/300
 - 0s - loss: 2.5370 - acc: 0.3636
Epoch 6/300
 - 0s - loss: 2.5268 - acc: 0.3636
Epoch 7/300
 - 0s - loss: 2.5165 - acc: 0.3636
Epoch 8/300
 - 0s - loss: 2.5058 - acc: 0.2727
Epoch 9/300
 - 0s - loss: 2.4949 - acc: 0.3636
Epoch 10/300
 - 0s - loss: 2.4836 - acc: 0.3636
Epoch 11/300
 - 0s - loss: 2.4719 - acc: 0.3636
Epoch 12/300
 - 0s - loss: 2.4598 - acc: 0.3636
Epoch 13/300
 - 0s - loss: 2.4473 - acc: 0.3636
Epoch 14/300
 - 0s - loss: 2.4342 - acc: 0.4545
Epoch 15/300
 - 0s - loss: 2.4207 - acc: 0.4545
Epoch 16/300
 - 0s - loss: 2.4067 - acc: 0.4545
Epoch 17/300
 - 0s - loss: 2.3920 - acc: 0.4545
Epoch 18/300
 - 0s - loss: 2.3769 - acc: 0.4545
Epoch 19/300
 - 0s - loss: 2.3611 - acc: 0.4545
Epoch 20/300
 - 0s - loss: 2.3448 - acc: 0.4545
Epoch 21/300
 - 0s - loss: 2.3279 - acc: 0.4545
E

Epoch 171/300
 - 0s - loss: 0.2285 - acc: 1.0000
Epoch 172/300
 - 0s - loss: 0.2253 - acc: 1.0000
Epoch 173/300
 - 0s - loss: 0.2222 - acc: 1.0000
Epoch 174/300
 - 0s - loss: 0.2192 - acc: 1.0000
Epoch 175/300
 - 0s - loss: 0.2162 - acc: 1.0000
Epoch 176/300
 - 0s - loss: 0.2133 - acc: 1.0000
Epoch 177/300
 - 0s - loss: 0.2104 - acc: 1.0000
Epoch 178/300
 - 0s - loss: 0.2076 - acc: 1.0000
Epoch 179/300
 - 0s - loss: 0.2049 - acc: 1.0000
Epoch 180/300
 - 0s - loss: 0.2022 - acc: 1.0000
Epoch 181/300
 - 0s - loss: 0.1995 - acc: 1.0000
Epoch 182/300
 - 0s - loss: 0.1969 - acc: 1.0000
Epoch 183/300
 - 0s - loss: 0.1943 - acc: 1.0000
Epoch 184/300
 - 0s - loss: 0.1918 - acc: 1.0000
Epoch 185/300
 - 0s - loss: 0.1893 - acc: 1.0000
Epoch 186/300
 - 0s - loss: 0.1869 - acc: 1.0000
Epoch 187/300
 - 0s - loss: 0.1845 - acc: 1.0000
Epoch 188/300
 - 0s - loss: 0.1821 - acc: 1.0000
Epoch 189/300
 - 0s - loss: 0.1798 - acc: 1.0000
Epoch 190/300
 - 0s - loss: 0.1775 - acc: 1.0000
Epoch 191/300
 - 0s 

In [53]:


def word_gen(model, t, curr_word, num):
    ini_word=curr_word
    sent=''
    #model, tokenizer, input_word, number
    for _ in range(num):
        encoded=t.texts_to_sequences([curr_word])[0]
        encoded=pad_sequences([encoded],maxlen=5, padding='pre')
        res=model.predict_classes(encoded)
        
        for word, index in t.word_index.items():
            if res==index: # 같은 단어를 찾았다면
                break
        curr_word=curr_word+' '+word
        sent=sent+" "+word
        
    sent=ini_word+sent
    return sent

print(word_gen(model, t, '멀티캠퍼스에', 3))

멀티캠퍼스에 있는 교육생들이 공부하고


In [56]:
import pandas as pd
df=pd.read_csv("nyt-comments/ArticlesApril2018.csv")



Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [57]:
print(df.columns)

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [60]:
headline=[]

headline.extend(list(df.headline.values))
headline=[n for n in headline if n !='Unknown']
len(headline)

1214

In [67]:
from string import punctuation
def mypreprocessing(s):# 점 제거, 소문자 변환
    return ''.join(ch for ch in s if ch not in punctuation)
    
text=[mypreprocessing(x) for x in headline]
print(text)

['Former NFL Cheerleaders’ Settlement Offer 1 and a Meeting With Goodell', 'EPA to Unveil a New Rule Its Effect Less Science in Policymaking', 'The New Noma Explained', 'How a Bag of Texas Dirt  Became a Times Tradition', 'Is School a Place for SelfExpression', 'Commuter Reprogramming', 'Ford Changed Leaders Looking for a Lift It’s Still Looking', 'Romney Failed to Win at Utah Convention But Few Believe He’s Doomed', 'Chain Reaction', 'He Forced the Vatican to Investigate Sex Abuse Now He’s Meeting With Pope Francis', 'In Berlin artists find a home', 'The Right Stuff', 'Jimmy Carter Knows What North Korea Wants', 'The Truth Is Out There', 'New Jersey Ruling Could Reignite Battle Over ChurchState Separation', 'Procrastinating', 'Word  Quiz dilatory', 'My LifeThreatening Bout With E Coli Food Poisoning', 'Choosing Brexit a Town Yearned for Its Seafaring Past and Muddied Its Future', 'A Quote Disproved', 'Hot Stuff Turns Cold', 'At the Top The Pay Gap May Be Gone', 'Years of Bizarre Behav

In [71]:
t=Tokenizer()
t.fit_on_texts(text)
vocab_size=len(t.word_index)+1
t.word_index


{'the': 1,
 'a': 2,
 'to': 3,
 'of': 4,
 'in': 5,
 'for': 6,
 'and': 7,
 'on': 8,
 'is': 9,
 'with': 10,
 'trump': 11,
 'as': 12,
 'at': 13,
 'new': 14,
 'from': 15,
 'how': 16,
 'it': 17,
 'an': 18,
 'be': 19,
 'that': 20,
 'season': 21,
 'us': 22,
 'you': 23,
 'what': 24,
 'episode': 25,
 'can': 26,
 'your': 27,
 'he': 28,
 'not': 29,
 'his': 30,
 'are': 31,
 'teaching': 32,
 'out': 33,
 'was': 34,
 'no': 35,
 'by': 36,
 'trump’s': 37,
 'has': 38,
 'war': 39,
 'over': 40,
 'may': 41,
 'into': 42,
 'more': 43,
 'now': 44,
 'why': 45,
 'who': 46,
 'about': 47,
 'recap': 48,
 'activities': 49,
 '1': 50,
 'we': 51,
 'do': 52,
 'women': 53,
 'when': 54,
 'syria': 55,
 'trade': 56,
 'it’s': 57,
 '2': 58,
 'or': 59,
 'will': 60,
 'i': 61,
 'have': 62,
 'just': 63,
 'but': 64,
 'home': 65,
 'up': 66,
 'this': 67,
 'one': 68,
 'off': 69,
 'house': 70,
 'president': 71,
 'gop': 72,
 'our': 73,
 'long': 74,
 'facebook': 75,
 'case': 76,
 'life': 77,
 'end': 78,
 'some': 79,
 'power': 80,
 'say'

In [72]:
sequences=[]

In [79]:
for line in text:
    encoded=t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence=encoded[:i+1]
        sequences.append(sequence)
print(sequences[:20])

[[95, 263], [95, 263, 1100], [95, 263, 1100, 1101], [95, 263, 1100, 1101, 572], [95, 263, 1100, 1101, 572, 50], [95, 263, 1100, 1101, 572, 50, 7], [95, 263, 1100, 1101, 572, 50, 7, 2], [95, 263, 1100, 1101, 572, 50, 7, 2, 365], [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10], [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10, 1102], [96, 3], [96, 3, 1103], [96, 3, 1103, 2], [96, 3, 1103, 2, 14], [96, 3, 1103, 2, 14, 573], [96, 3, 1103, 2, 14, 573, 85], [96, 3, 1103, 2, 14, 573, 85, 1104], [96, 3, 1103, 2, 14, 573, 85, 1104, 366], [96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367], [96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367, 5]]


In [80]:
max(len(s) for s in sequences)

24